
# 1  Data Loading, Cleaning, Labelling, and Exploration<a class="anchor" id="1"></a>
## 1.1 Data loading<a class="anchor" id="1.1"></a>

In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import *
import os
from pyspark.sql import Window
import matplotlib.pyplot as plt
from pyspark.sql.types import *
from pyspark.sql import Row
import time
import numpy as np

master = "local[*]"
app_name = "Assignment 2 - Morad Abou Shadi (29799260)"
spark_conf = SparkConf().setMaster(master).setAppName(app_name)

from pyspark import SparkContext # Spark
spark = SparkSession.builder.config(conf=spark_conf).getOrCreate()
sc = spark.sparkContext
sc.setLogLevel('ERROR')

In [2]:
flightsRawDf = spark.read.option("header",True).option("inferSchema",True).format("csv").load("flight*.csv")
flightsRawDf.count()

582184

In [3]:
allColNameFlights = flightsRawDf.columns
print(allColNameFlights)

['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'FLIGHT_NUMBER', 'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT', 'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'WHEELS_ON', 'TAXI_IN', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME', 'ARRIVAL_DELAY', 'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON', 'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY']


## 1.2 Data cleaning<a class="anchor" id="1.1"></a>

In [4]:
flightsRawDf.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in flightsRawDf.columns[0:10]]).show()
flightsRawDf.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in flightsRawDf.columns[10:20]]).show()
flightsRawDf.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in flightsRawDf.columns[20:28]]).show()
flightsRawDf.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in flightsRawDf.columns[28:31]]).show()

+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+
|   0|    0|  0|          0|      0|            0|       1462|             0|                  0|                  0|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+

+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+
|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|
+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+
|          8633|           8633|    8891|      8891|      

In [5]:
rowsToDrop =  ['CANCELLATION_REASON','AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY','LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY']
flightsRawDf = flightsRawDf.drop(*rowsToDrop)
print(len(flightsRawDf.columns))

25


In [6]:
flightsDf = flightsRawDf.na.drop()
flightsDf.count()

571729

## 1.3 Data Labelling<a class="anchor" id="1.1"></a>

## 1.3.1 (a) binaryArrDelay and binaryDeptDelay <a class="anchor" id="1.1"></a>

In [7]:
flightsDf = flightsDf.withColumn("binaryArrDelay",F.when((F.col("ARRIVAL_DELAY") > 0),1).otherwise(0))
flightsDf = flightsDf.withColumn("binaryDeptDelay",F.when((F.col("DEPARTURE_DELAY") > 0),1).otherwise(0))

## 1.3.1 (b) multiClassArrDelay and multiClassDeptDelay <a class="anchor" id="1.1"></a>

In [8]:
flightsDf = flightsDf.withColumn("multiClassArrDelay",F.when((F.col("ARRIVAL_DELAY") < 5),0).when((F.col("ARRIVAL_DELAY").between(5,20)),1).otherwise(2))
flightsDf = flightsDf.withColumn("multiClassDeptDelay",F.when((F.col("DEPARTURE_DELAY") < 5),0).when((F.col("DEPARTURE_DELAY").between(5,20)),1).otherwise(2))

## 1.4 Data Exploration / Exploratory Analysis<a class="anchor" id="1.1"></a>

In [9]:
columnList = [field.name for field in flightsRawDf.schema.fields if isinstance(field.dataType, IntegerType)]
flightsDf.select(columnList[0:6]).summary().show()
flightsDf.select(columnList[6:12]).summary().show()
flightsDf.select(columnList[12:18]).summary().show()
flightsDf.select(columnList[18:24]).summary().show()

+-------+------+------------------+------------------+------------------+------------------+-------------------+
|summary|  YEAR|             MONTH|               DAY|       DAY_OF_WEEK|     FLIGHT_NUMBER|SCHEDULED_DEPARTURE|
+-------+------+------------------+------------------+------------------+------------------+-------------------+
|  count|571729|            571729|            571729|            571729|            571729|             571729|
|   mean|2015.0| 6.542260406591235|15.711267051347754|3.9328580498802754|2160.6212803618496|  1328.704886405972|
| stddev|   0.0|3.3989080609437297| 8.768837723805682|1.9840872105208625|1752.1910605700507| 483.32212195085305|
|    min|  2015|                 1|                 1|                 1|                 1|                  1|
|    25%|  2015|                 4|                 8|                 2|               727|                917|
|    50%|  2015|                 7|                16|                 4|              1678|    

In [10]:
flightsDf.groupBy("binaryArrDelay").count().show()
flightsDf.groupBy("binaryDeptDelay").count().show()
flightsDf.groupBy("multiClassArrDelay").count().show()
flightsDf.groupBy("multiClassDeptDelay").count().show()

+--------------+------+
|binaryArrDelay| count|
+--------------+------+
|             1|209055|
|             0|362674|
+--------------+------+

+---------------+------+
|binaryDeptDelay| count|
+---------------+------+
|              1|211228|
|              0|360501|
+---------------+------+

+------------------+------+
|multiClassArrDelay| count|
+------------------+------+
|                 1| 82428|
|                 2| 85692|
|                 0|403609|
+------------------+------+

+-------------------+------+
|multiClassDeptDelay| count|
+-------------------+------+
|                  1| 78843|
|                  2| 84831|
|                  0|408055|
+-------------------+------+



- For both binaryArrDelay and binaryDeptDelay it can be seen that the number of not late entries are roughly 1.7x that of the late entries.
- multiClassArrDelay has a relative balance between labels 1 and 2, but are both significantly less than label 0.
- multiClassDeptDelay is very unbalanced as the count of label 0 is 10x that of label 1, while label 2 is somewhere in between.
- Most machine learning algorithms used for classification were designed around the assumption of an equal number of examples for each class. As we have clearly stated in the aforementioned points, the data is rather unbalanced in both binary and multiclass classification; in turn, this will lead to poor performance and accuracy in predictions, especially for the minority class/classes.
- We can conclude that generally, the flights are early on departure and arrival.

# 2  Feature extraction and ML Training<a class="anchor" id="1"></a>
## 2.1 Prepare the feature columns<a class="anchor" id="1.1"></a>

In [11]:
flightsDf = flightsDf.withColumn("DEPT_TIME_FLAG",F.when((F.col("SCHEDULED_DEPARTURE").between(0,459 )),"Midnight").when((F.col("SCHEDULED_DEPARTURE").between(500,1059)),"Morning").when((F.col("SCHEDULED_DEPARTURE").between(1100,1559)),"Afternoon").when((F.col("SCHEDULED_DEPARTURE").between(1600,1959)),"Evening").otherwise("Night"))

In [12]:
newCols = ['MONTH', 'DAY','DAY_OF_WEEK', 'AIRLINE', 'DEPT_TIME_FLAG','ELAPSED_TIME','DISTANCE', 'TAXI_IN', 'TAXI_OUT', 'AIR_TIME','SCHEDULED_TIME', 'binaryArrDelay', 'binaryDeptDelay', 'multiClassDeptDelay','multiClassArrDelay']
flightsDf_data = flightsDf.select(newCols)

## 2.2 Preparing any Spark ML Transformers/ Estimators for features and models<a class="anchor" id="1.1"></a>

## 2.2.1 Transformers/Estimators<a class="anchor" id="1.1"></a>

In [13]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

numericInputs = ['MONTH', 'DAY','DAY_OF_WEEK','ELAPSED_TIME','DISTANCE', 'TAXI_IN', 'TAXI_OUT', 'AIR_TIME','SCHEDULED_TIME']

inputs_OHE = ["AIRLINE_index","DEPT_TIME_FLAG_index"]
outputCols_OHE = [f'{x}_vec' for x in inputs_OHE]

indexer = StringIndexer(inputCols=["AIRLINE","DEPT_TIME_FLAG"], outputCols=["AIRLINE_index","DEPT_TIME_FLAG_index"]).setHandleInvalid("skip")

encoder = OneHotEncoder(inputCols=inputs_OHE, outputCols=outputCols_OHE)

assembler = VectorAssembler(inputCols=outputCols_OHE + numericInputs , outputCol="features")

## 2.2.2 Decision Tree and Gradient Boosted Tree model<a class="anchor" id="1.1"></a>

In [14]:
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import GBTClassifier

dtArr = DecisionTreeClassifier(featuresCol = 'features', labelCol = 'binaryArrDelay')
dtDept = DecisionTreeClassifier(featuresCol = 'features', labelCol = 'binaryDeptDelay')
gbtArr = GBTClassifier(featuresCol = 'features', labelCol = 'binaryArrDelay')
gbtDept = GBTClassifier(featuresCol = 'features', labelCol = 'binaryDeptDelay')

## 2.2.3 Naive Bayes<a class="anchor" id="1.1"></a>

In [15]:
from pyspark.ml.classification import NaiveBayes


nbArr = NaiveBayes(labelCol = 'multiClassArrDelay', smoothing=1.0, modelType="multinomial",featuresCol = 'features')

## 2.2.4 Pipelines<a class="anchor" id="1.1"></a>

In [16]:
from pyspark.ml import Pipeline

stage_1 = indexer
stage_2 = encoder
stage_3 = assembler

pipelineDtArr = Pipeline(stages=[stage_1, stage_2, stage_3,dtArr])
pipelineDtDept = Pipeline(stages=[stage_1, stage_2, stage_3,dtDept])
pipelineGbtArr = Pipeline(stages=[stage_1, stage_2, stage_3,gbtArr])
pipelineGbtDept = Pipeline(stages=[stage_1, stage_2, stage_3,gbtDept])
pipelineNbArr = Pipeline(stages=[stage_1,stage_2, stage_3,nbArr])

## 2.3 Preparing the training and testing data<a class="anchor" id="1.1"></a>

In [17]:
train, test = flightsDf_data.randomSplit([0.8, 0.2])

## 2.4 Training and evaluating models<a class="anchor" id="1.1"></a>

## 2.4.1 (a & b) <a class="anchor" id="1.1"></a>

In [18]:
dtArrModel = pipelineDtArr.fit(train)
dtArrPrediction = dtArrModel.transform(test)
print("Decision tree model\n")
dtArrPrediction.groupBy(col("binaryArrDelay").alias("late (arrival)"),"prediction").count().show()

dtDeptModel = pipelineDtDept.fit(train)
dtDeptPrediction = dtDeptModel.transform(test)
dtDeptPrediction.groupBy(col("binaryDeptDelay").alias("late (departure)"),"prediction").count().show()
print("###########################################################\n")
print("Gradient Boosted Tree model\n")
gbtArrModel = pipelineGbtArr.fit(train)
gbtArrPrediction = gbtArrModel.transform(test)
gbtArrPrediction.groupBy(col("binaryArrDelay").alias("late (arrival)"),"prediction").count().show()

gbtDeptModel = pipelineGbtDept.fit(train)
gbtDeptPrediction = gbtDeptModel.transform(test)
gbtDeptPrediction.groupBy(col("binaryDeptDelay").alias("late (departure)"),"prediction").count().show()

Decision tree model

+--------------+----------+-----+
|late (arrival)|prediction|count|
+--------------+----------+-----+
|             1|       0.0|29880|
|             0|       0.0|67865|
|             1|       1.0|11800|
|             0|       1.0| 4774|
+--------------+----------+-----+

+----------------+----------+-----+
|late (departure)|prediction|count|
+----------------+----------+-----+
|               1|       0.0|32551|
|               0|       0.0|65352|
|               1|       1.0| 9779|
|               0|       1.0| 6637|
+----------------+----------+-----+

###########################################################

Gradient Boosted Tree model

+--------------+----------+-----+
|late (arrival)|prediction|count|
+--------------+----------+-----+
|             1|       0.0|26137|
|             0|       0.0|67814|
|             1|       1.0|15543|
|             0|       1.0| 4825|
+--------------+----------+-----+

+----------------+----------+-----+
|late (departure)|

## 2.4.1 (c) AUC, accuracy, recall, and precision <a class="anchor" id="1.1"></a>

In [19]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction").setLabelCol("binaryArrDelay")
eval_accuracy = MulticlassClassificationEvaluator(labelCol="binaryArrDelay", predictionCol="prediction", metricName="accuracy")
eval_precision = MulticlassClassificationEvaluator(labelCol="binaryArrDelay", predictionCol="prediction", metricName="precisionByLabel")
eval_recall = MulticlassClassificationEvaluator(labelCol="binaryArrDelay", predictionCol="prediction", metricName="recallByLabel")

auc_dtArrPrediction = evaluator.evaluate(dtArrPrediction)
accuracy_dtArrPrediction = eval_accuracy.evaluate(dtArrPrediction)
precision_dtArrPrediction = eval_precision.evaluate(dtArrPrediction)
recall_dtArrPrediction = eval_recall.evaluate(dtArrPrediction)
print("AUC for arrival delay decision tree: ", auc_dtArrPrediction)
print("Accuracy for arrival delay decision tree: ", accuracy_dtArrPrediction)
print("Precision for arrival delay decision tree: ", precision_dtArrPrediction)
print("Recall for arrival delay decision tree: ", recall_dtArrPrediction)
print("##################################################################################\n")

auc_gbtArrPrediction = evaluator.evaluate(gbtArrPrediction)
accuracy_gbtArrPrediction = eval_accuracy.evaluate(gbtArrPrediction)
precision_gbtArrPrediction = eval_precision.evaluate(gbtArrPrediction)
recall_gbtArrPrediction = eval_recall.evaluate(gbtArrPrediction)
print("AUC for arrival delay gradient boosted tree: ", auc_gbtArrPrediction)
print("Accuracy for arrival gradient boosted tree: ", accuracy_gbtArrPrediction)
print("Precision for arrival gradient boosted tree: ", precision_gbtArrPrediction)
print("Recall for arrival delay gradient boosted tree: ", recall_gbtArrPrediction)
print("##################################################################################\n")

evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction").setLabelCol("binaryDeptDelay")
eval_accuracy = MulticlassClassificationEvaluator(labelCol="binaryDeptDelay", predictionCol="prediction", metricName="accuracy")
eval_precision = MulticlassClassificationEvaluator(labelCol="binaryDeptDelay", predictionCol="prediction", metricName="precisionByLabel")
eval_recall = MulticlassClassificationEvaluator(labelCol="binaryDeptDelay", predictionCol="prediction", metricName="recallByLabel")

auc_dtDeptPrediction = evaluator.evaluate(dtDeptPrediction)
accuracy_dtDeptPrediction = eval_accuracy.evaluate(dtDeptPrediction)
precision_dtDeptPrediction = eval_precision.evaluate(dtDeptPrediction)
recall_dtDeptPrediction = eval_recall.evaluate(dtDeptPrediction)
print("AUC for departure delay decision tree: ", auc_dtDeptPrediction)
print("Accuracy for departure delay decision tree: ", accuracy_dtDeptPrediction)
print("Precision for departure delay decision tree: ", precision_dtDeptPrediction)
print("Recall for departure delay decision tree: ", recall_dtDeptPrediction)
print("##################################################################################\n")

auc_gbtDeptPrediction = evaluator.evaluate(gbtDeptPrediction)
accuracy_gbtDeptPrediction = eval_accuracy.evaluate(gbtDeptPrediction)
precision_gbtDeptPrediction = eval_precision.evaluate(gbtDeptPrediction)
recall_gbtDeptPrediction = eval_recall.evaluate(gbtDeptPrediction)
print("AUC for departure delay gradient boosted tree: ", auc_gbtDeptPrediction)
print("Accuracy for departure gradient boosted tree: ", accuracy_gbtDeptPrediction)
print("Precision for departure gradient boosted tree: ", precision_gbtDeptPrediction)
print("Recall for departure delay gradient boosted tree: ", recall_gbtDeptPrediction)
print("##################################################################################\n")

AUC for arrival delay decision tree:  0.3982553746184528
Accuracy for arrival delay decision tree:  0.6968657878392918
Precision for arrival delay decision tree:  0.6943066141490614
Recall for arrival delay decision tree:  0.9342777295942951
##################################################################################

AUC for arrival delay gradient boosted tree:  0.7604217119278281
Accuracy for arrival gradient boosted tree:  0.7291613817475661
Precision for arrival gradient boosted tree:  0.721801790294941
Recall for arrival delay gradient boosted tree:  0.9335756274177783
##################################################################################

AUC for departure delay decision tree:  0.4565384698951811
Accuracy for departure delay decision tree:  0.6572048390906149
Precision for departure delay decision tree:  0.6675178492998172
Recall for departure delay decision tree:  0.9078053591520927
###############################################################################

## 2.4.1 (d) Discuss which is the better model based on performance metric above<a class="anchor" id="1.1"></a>

1. Based on the above readings, we can see that the gradient boosted trees have significantly greater AUC values than the decision trees.
2. It can also be seen that the gradient boosted models have higher accuracy and precision for the specific classifications of arrival/departure delays.
3. The recall value are very close between both models and is generall higher for the decision trees.
4. Based on the above disccusion, it is safe to conclude that the gradient boosted models offer a better performance in both classifcation tasks.

In [20]:
arrival_delay_prediction_model = gbtArrModel
departure_delay_prediction_model = gbtDeptModel

## 2.4.1 (e) Tree structure, and the top-3 features with each corresponding feature importance <a class="anchor" id="1.1"></a>

In [21]:
import pandas as pd
def ExtractFeatureImp(featureImp, dataset, featuresCol):
    list_extract = []
    for i in dataset.schema[featuresCol].metadata["ml_attr"]["attrs"]:
        list_extract = list_extract + dataset.schema[featuresCol].metadata["ml_attr"]["attrs"][i]
    varlist = pd.DataFrame(list_extract)
    varlist['score'] = varlist['idx'].apply(lambda x: featureImp[x])
    return(varlist.sort_values('score', ascending = False))

# NOTE: the above function was obtained from the hint provided in the assignment briefing presentation --> https://www.timlrx.com/blog/feature-selection-using-feature-importance-score-creating-a-pyspark-estimator

In [22]:
print("Decision tree for arrival delay\n")
print(dtArrModel.stages[-1].toDebugString)
display(ExtractFeatureImp(dtArrModel.stages[-1].featureImportances, dtArrPrediction, "features").head(3))

Decision tree for arrival delay

DecisionTreeClassificationModel: uid=DecisionTreeClassifier_c2790794178b, depth=5, numNodes=21, numClasses=2, numFeatures=26
  If (feature 23 <= 24.5)
   If (feature 22 <= 19.5)
    If (feature 13 in {1.0})
     Predict: 0.0
    Else (feature 13 not in {1.0})
     If (feature 23 <= 17.5)
      Predict: 0.0
     Else (feature 23 > 17.5)
      If (feature 0 in {1.0})
       Predict: 1.0
      Else (feature 0 not in {1.0})
       Predict: 0.0
   Else (feature 22 > 19.5)
    Predict: 1.0
  Else (feature 23 > 24.5)
   If (feature 23 <= 33.5)
    If (feature 25 <= 160.5)
     Predict: 1.0
    Else (feature 25 > 160.5)
     If (feature 20 <= 170.5)
      Predict: 0.0
     Else (feature 20 > 170.5)
      Predict: 1.0
   Else (feature 23 > 33.5)
    If (feature 25 <= 154.5)
     Predict: 1.0
    Else (feature 25 > 154.5)
     If (feature 20 <= 162.5)
      Predict: 0.0
     Else (feature 20 > 162.5)
      Predict: 1.0



idx                              name     score
6    23                          TAXI_OUT  0.694282
5    22                           TAXI_IN  0.145427
22   13  DEPT_TIME_FLAG_index_vec_Morning  0.117039

In [23]:
print("Decision tree for departure delay\n")
print(dtDeptModel.stages[-1].toDebugString)
display(ExtractFeatureImp(dtDeptModel.stages[-1].featureImportances, dtDeptPrediction, "features").head(3))

Decision tree for departure delay

DecisionTreeClassificationModel: uid=DecisionTreeClassifier_3990362fca82, depth=5, numNodes=43, numClasses=2, numFeatures=26
  If (feature 13 in {1.0})
   If (feature 5 in {1.0})
    If (feature 21 <= 1242.5)
     If (feature 17 <= 8.5)
      Predict: 0.0
     Else (feature 17 > 8.5)
      If (feature 23 <= 30.5)
       Predict: 0.0
      Else (feature 23 > 30.5)
       Predict: 1.0
    Else (feature 21 > 1242.5)
     If (feature 17 <= 8.5)
      If (feature 23 <= 16.5)
       Predict: 0.0
      Else (feature 23 > 16.5)
       Predict: 1.0
     Else (feature 17 > 8.5)
      Predict: 0.0
   Else (feature 5 not in {1.0})
    If (feature 20 <= 181.5)
     Predict: 0.0
    Else (feature 20 > 181.5)
     If (feature 23 <= 33.5)
      Predict: 0.0
     Else (feature 23 > 33.5)
      If (feature 0 in {1.0})
       Predict: 1.0
      Else (feature 0 not in {1.0})
       Predict: 0.0
  Else (feature 13 not in {1.0})
   If (feature 0 in {1.0})
    If (feature 1

idx                              name     score
22   13  DEPT_TIME_FLAG_index_vec_Morning  0.497106
9     0              AIRLINE_index_vec_WN  0.188619
14    5              AIRLINE_index_vec_UA  0.167606

In [24]:
print("Gradient boosted tree for arrival delay\n")
print(gbtArrModel.stages[-1].toDebugString)
display(ExtractFeatureImp(gbtArrModel.stages[-1].featureImportances, gbtArrPrediction, "features").head(3))

Gradient boosted tree for arrival delay

GBTClassificationModel: uid = GBTClassifier_47f1fa50cd0c, numTrees=20, numClasses=2, numFeatures=26
  Tree 0 (weight 1.0):
    If (feature 23 <= 24.5)
     If (feature 22 <= 19.5)
      If (feature 13 in {1.0})
       If (feature 23 <= 16.5)
        If (feature 22 <= 8.5)
         Predict: -0.6563243228550849
        Else (feature 22 > 8.5)
         Predict: -0.5213573330154567
       Else (feature 23 > 16.5)
        If (feature 22 <= 8.5)
         Predict: -0.39739836193158656
        Else (feature 22 > 8.5)
         Predict: -0.18459579885423297
      Else (feature 13 not in {1.0})
       If (feature 23 <= 17.5)
        If (feature 0 in {0.0})
         Predict: -0.40851446215381704
        Else (feature 0 not in {0.0})
         Predict: -0.1647908260777235
       Else (feature 23 > 17.5)
        If (feature 0 in {0.0})
         Predict: -0.14571440305556696
        Else (feature 0 not in {0.0})
         Predict: 0.27349624060150374
     Else (

idx            name     score
8   25  SCHEDULED_TIME  0.260380
3   20    ELAPSED_TIME  0.227466
6   23        TAXI_OUT  0.137267

In [25]:
print("Gradient boosted tree for departure delay\n")
print(gbtDeptModel.stages[-1].toDebugString)
display(ExtractFeatureImp(gbtDeptModel.stages[-1].featureImportances, gbtDeptPrediction, "features").head(3))

Gradient boosted tree for departure delay

GBTClassificationModel: uid = GBTClassifier_861c30d21c1f, numTrees=20, numClasses=2, numFeatures=26
  Tree 0 (weight 1.0):
    If (feature 13 in {1.0})
     If (feature 5 in {0.0})
      If (feature 20 <= 177.5)
       If (feature 23 <= 28.5)
        If (feature 10 in {0.0})
         Predict: -0.568027696141372
        Else (feature 10 not in {0.0})
         Predict: -0.3132295719844358
       Else (feature 23 > 28.5)
        If (feature 17 <= 3.5)
         Predict: -0.3046181172291297
        Else (feature 17 > 3.5)
         Predict: -0.42882288228822885
      Else (feature 20 > 177.5)
       If (feature 23 <= 33.5)
        If (feature 0 in {0.0})
         Predict: -0.4671016856987493
        Else (feature 0 not in {0.0})
         Predict: -0.3360924297292637
       Else (feature 23 > 33.5)
        If (feature 23 <= 70.5)
         Predict: -0.25231719876416064
        Else (feature 23 > 70.5)
         Predict: 0.12264150943396226
     Else (f

idx                              name     score
0    17                             MONTH  0.251217
1    18                               DAY  0.250912
22   13  DEPT_TIME_FLAG_index_vec_Morning  0.091262

## 2.4.2 (a)  <a class="anchor" id="1.1"></a>

In [26]:
nbArrModel = pipelineNbArr.fit(train)
nbArrPrediction = nbArrModel.transform(test)

## 2.4.2 (b)  <a class="anchor" id="1.1"></a>

In [27]:
nbArrPrediction.groupBy(col("multiClassArrDelay").alias("late (arrival)"),"prediction").count().show()

+--------------+----------+-----+
|late (arrival)|prediction|count|
+--------------+----------+-----+
|             2|       1.0|  337|
|             1|       0.0| 8214|
|             0|       2.0|25741|
|             0|       0.0|54034|
|             1|       2.0| 7692|
|             2|       2.0| 8332|
|             1|       1.0|  489|
|             2|       0.0| 8513|
|             0|       1.0|  967|
+--------------+----------+-----+



## 2.4.2 (c) AUC, accuracy, recall, and precision <a class="anchor" id="1.1"></a>

In [28]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction").setLabelCol("multiClassArrDelay")
eval_accuracy = MulticlassClassificationEvaluator(labelCol="multiClassArrDelay", predictionCol="prediction", metricName="accuracy")
eval_precision = MulticlassClassificationEvaluator(labelCol="multiClassArrDelay", predictionCol="prediction", metricName="precisionByLabel")
eval_recall = MulticlassClassificationEvaluator(labelCol="multiClassArrDelay", predictionCol="prediction", metricName="recallByLabel")

auc_nbArrPrediction = evaluator.evaluate(nbArrPrediction)
accuracy_nbArrPrediction = eval_accuracy.evaluate(nbArrPrediction)
precision_nbArrPrediction = eval_precision.evaluate(nbArrPrediction)
recall_nbArrPrediction = eval_recall.evaluate(nbArrPrediction)
print("AUC for arrival delay naive bayes: ", auc_nbArrPrediction)
print("Accuracy for arrival delay naive bayes: ", accuracy_nbArrPrediction)
print("Precision for arrival delay naive bayes: ", precision_nbArrPrediction)
print("Recall for arrival delay naive bayes: ", recall_nbArrPrediction)

AUC for arrival delay naive bayes:  0.46720827258646996
Accuracy for arrival delay naive bayes:  0.5498211146003726
Precision for arrival delay naive bayes:  0.7636127245233957
Recall for arrival delay naive bayes:  0.6692180030219712


## 2.4.2 (d) Discuss which metric is more proper for measuring the model performance on predicting early/on-time/late events, in order to give the performers good recommendations.
<a class="anchor" id="1.1"></a>

- Generally speaking, the accuracy metric is often the most looked at performance metric when it comes to multiclass classification as it involves more complexity than simply fragmenting the predictions into a binary classification. 
- However, looking at our data set, we can clearly see that it is highly imbalanced. When data is highly imbalanced, we often need to neglect the accuracy metric. This is because a high accuracy value may stem from the model correctly prediciting one of the majorty classes in the classification and completely ignores the minorities. So we skip the accuracy metric.
- Similar to accuracy, AUC values are also often very senstivie to highly skewed data and can be better optimised depending on how the labelling is set up. This is however too sesntive to be reliable and therefore we skip AUC aswell.
- Oftentimes, the main metrics considered when dealing with imbalanced data are Recall and Precision combined as an F1 score. For this scenario, I would use recall and precision as the most proper metrics to use for measuring the performance for our data set.

## 2.4.2 (e) Discuss the ways the classification performance can be improved<a class="anchor" id="1.1"></a>

1. Considering that correlated features are counted twice, it can really skew the data. Hence, we can remove correlated data and use independant features instead.
2. Apply a classifier combination technique (eg, ensembling, boosting, bagging).
3. Classificaiton models work with probabilities and these are often small numbers, when operations are done on smaller number, they usually get smaller which makes it more difficult for the model to work with. A way to overcome this would be to use log probability.
4. Using a large dataset may throw off the model, only using 'enough' data to train the model is ideal.